# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
from unityagents import UnityEnvironment


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [3]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)


# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[1])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [8]:
for i in range(1, 10):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, scores))

Score (max over agents) from episode 1: [ 0.   -0.01]
Score (max over agents) from episode 2: [-0.01  0.  ]
Score (max over agents) from episode 3: [0.   0.09]
Score (max over agents) from episode 4: [-0.01  0.  ]
Score (max over agents) from episode 5: [ 0.   -0.01]
Score (max over agents) from episode 6: [ 0.   -0.01]
Score (max over agents) from episode 7: [-0.01  0.1 ]
Score (max over agents) from episode 8: [ 0.   -0.01]
Score (max over agents) from episode 9: [ 0.   -0.01]


When finished, you can close the environment.

In [28]:
env.close()

UnityEnvironmentException: No Unity environment is loaded.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
import random
import copy
from collections import namedtuple, deque

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [7]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1./np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    
    def __init__(self, state_size, action_size, seed, fc1_units= 256, fc2_units= 128):
        
        
        
        
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size*2, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        
        self.reset_parameters()
        
        
        
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3,3e-3)
        
    def forward(self, state):
        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))
    
    
class Critic(nn.Module):
    
    def __init__(self, state_size, action_size, seed, fc1_units = 256, fc2_units= 128):
        
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size*2, fc1_units)
        self.fc2 = nn.Linear(fc1_units+action_size*2, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()
    
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
        
    def forward(self, state, action):
        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(torch.cat((x, action), dim = 1)))
        return self.fc3(x)
    

    


In [20]:
class Agent():
    
    def __init__(self, state_size, action_size, num_agents, random_seed):
        
        self.state_size = state_size
        self.action_size = action_size
        self.num_agents = num_agents
        self.seed = random.seed(random_seed)
        self.t_step = 0
        
        
        #Actor Network
        
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr = LR_ACTOR)
        
        
        #critic network
        
        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
        
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr = LR_CRITIC, weight_decay = WEIGHT_DECAY)
        
        for target_param, local_param in zip(self.actor_target.parameters(), self.actor_local.parameters()):
            target_param.data.copy_(local_param.data)
        
        for target_param, local_param in zip(self.critic_target.parameters(), self.critic_local.parameters()):
            target_param.data.copy_(local_param.data)
        
        
        #noise process
        
        self.noise = OUNoise((num_agents, action_size), random_seed)
        
        #prioritized replay memory
        
        self.memory = PrioritizedReplay(BATCH_SIZE, BUFFER_SIZE, random_seed, ALPHA)
        
    def step(self, state, action, reward, next_state, done, agent_number, beta):
        
        self.t_step +=1
        self.append_sample( state, action, reward, next_state, done, agent_number)
        
        
        if self.memory.ready():
            if self.t_step % UPDATE_EVERY == 0:
                for _ in range(N_UPDATES):
                    experiences = self.memory.sample(beta)
                    self.learn(experiences, GAMMA, agent_number)
    
    
    def append_sample(self, state, action, reward, next_state, done, agent_number):
        
        self.actor_target.eval()
        self.critic_target.eval()
        self.critic_local.eval()
        
        state = torch.from_numpy(state).float().to(device)
        next_state = torch.from_numpy(next_state).float().to(device)
        action =torch.from_numpy(action).float().to(device)
        
        with torch.no_grad():
            
            actions_next = self.actor_target.forward(state)
            
            if agent_number == 0:
                
                actions_next = torch.cat((actions_next, action[:,2:]), dim=1)
                
            elif agent_number == 1:
                
                actions_next = torch.cat((action[:,:2], actions_next), dim =1)
            
            
            Q_targets_next = self.critic_target.forward(next_state, actions_next)
            Q_targets = reward + GAMMA*Q_targets_next*(1-done)       
            Q_expected = self.critic_local(state, action)
            
            
        error = (Q_expected - Q_targets)**2
        
        self.memory.add(state, action, reward, next_state, done, error)
        
        self.critic_local.train()
        self.actor_target.train()
        self.critic_target.train()
        
        
    def act(self, states, add_noise):
        
        states = torch.from_numpy(states).float().to(device)
        actions = np.zeros((self.num_agents, self.action_size))
        self.actor_local.eval()
        
        with torch.no_grad():
            for agent_num, state in enumerate(states):
                action = self.actor_local(state).cpu().data.numpy()
                actions[agent_num, :] = action
            
        self.actor_local.train()
        
        if add_noise:
            
            actions += eps* self.noise.sample()
            
        return np.clip(actions, -1, 1)
    
    def reset(self):
        self.noise.reset()
        
        
    def learn(self, experiences, gamma, agent_number):
        
        
        states, actions, rewards, next_states, dones, is_weight, idxs = experiences
        
        
        
        actions_next =self.actor_target.forward(next_states)
        
        if agent_number == 0:
            actions_next = torch.cat((actions_next, actions[:,2:]), dim =1)
        else:
            actions_next = torch.cat((actions[:,:2], actions_next), dim = 1)
        
        Q_targets_next = self.critic_target.forward(next_states, actions_next)
        
        Q_targets = rewards + (gamma*Q_targets_next*(1-dones))
        
        Q_expected = self.critic_local(states, actions)
        
        loss = (Q_expected - Q_targets).pow(2).reshape(is_weight.shape)*is_weight
        
        self.memory.update(idxs, loss.data.cpu().numpy())
        
        
        #Minimize critic loss
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        #Minimize actor loss
        actions_pred = self.actor_local.forward(states)
        
        if agent_number == 0:
            actions_pred = torch.cat((actions_pred, actions[:,2:]), dim = 1)
        else:
            actions_pred = torch.cat((actions[:,:2], actions_pred), dim = 1)
        
        actor_loss = -self.critic_local.forward(states, actions_pred).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local,  self.actor_target, TAU)
        
        
        
        
        
        
    def soft_update(self, local_model, target_model, tau):
        
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data+(1-tau)*target_param.data)
            
class OUNoise:
    
    def __init__(self, size, seed , mu=0.0, theta =0.13, sigma=0.2):
        
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.size = size
        self.reset()
        
        
    def reset(self):
        
        self.state= copy.copy(self.mu)
        
    
    def sample(self):
        
        x = self.state
        dx = self.theta * (self.mu-x) +self.sigma * np.random.standard_normal(self.size)
        self.state = x + dx
        
        return self.state
   


In [9]:
class ReplayBuffer:
    
    def __init__(self, batch_size, buffer_size, seed):
        
        self.memory = deque(maxlen = buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names = ["state", "actoion", "reward", "next_state", "done"] )
        
        self.seed = random.seed(seed)
        self.alpha = alpha
        self.epsilon = 1e-5
        
    def add(self, state, action, reward, next_state, done):
        
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
        
    
    def sample(self, beta):
        
        experiences = random.sample(self.memory, k =self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in expereinces if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None])).float().to(device)
        
        return(states, actions, rewards, next_states, dones)
    
    def ready(self):
        
        return len(self.memory)>= self.batch_size

    
    


In [10]:
class PrioritizedReplay:
    
    def __init__(self, batch_size, buffer_size, seed, alpha):
        
        
        self.memory = deque(maxlen = buffer_size)
        self.priorities = deque(maxlen = buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names = ["state", "action", "reward", "next_state", "done"] )
        
        self.seed = random.seed(seed)
        self.alpha = alpha
        self.epsilon = 1e-5
        
        
    def add(self, state, action, reward, next_state, done, priority):
        
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
        self.priorities.append(priority)
        
    def sample(self, beta):
        
        priorities = np.array(self.priorities).3reshape(-1)
        priorities = np.power(priorities + self.epsilon, self.alpha)
        p = priorities/np.sum(priorities)
        sampled_indices = np.random.choice(np.arange(len(p)), size = self.batch_size, p=p)
        experiences = [self.memory[i] for i in sampled_indices]
        p = np.array([p[i] for i in sampled_indices]).reshape(-1)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        weights = np.power(len(experiences)*p, -beta)
        weights /= weights.max()
        weights = torch.from_numpy(weights).float().to(device)
        
        return (states, actions, rewards, next_states, dones, weights, sampled_indices)
        
    def update(self, indices, priorities):
        
        for i, priority in zip(indices, priorities):
            self.priorities[i] = priority
        
    def ready(self):
        
        return len(self.memory) >= self.batch_size   


In [21]:
BATCH_SIZE = 128
BUFFER_SIZE =int(1e6)
GAMMA = 0.99
LR_ACTOR = 1e-3
LR_CRITIC = 1e-3
TAU = 6e-2
WEIGHT_DECAY = 0
UPDATE_EVERY =1
N_UPDATES = 1
ALPHA = 0
BETA_START = 0

ADD_NOISE = True
NOISE_START = 6
NOISE_END = 0
NOISE_DECAY = 500

device = torch.device( "cpu")

agent_0 = Agent(state_size, action_size, 1 , random_seed = 0)
agent_1 = Agent(state_size, action_size, 1 , random_seed = 0)




In [29]:
#test codes

n_episodes = 2000
states = env_info.vector_observations
states = np.reshape(states, (1,48))
agent_0.reset()
agent_1.reset()
scores = np.zeros(num_agents)


beta_schedule = lambda episode: min(1.0, BETA_START + episode * (1.0)/ n_episodes)
noise_schedule = lambda episode: max(NOISE_END, NOISE_START - episode * ((NOISE_START - NOISE_END) / NOISE_DECAY ))
beta = beta_schedule(1)
eps = noise_schedule(1)

action_0 = agent_0.act(states, ADD_NOISE)
action_1 = agent_0.act(states, ADD_NOISE)
actions =np.concatenate((action_0, action_1), axis =0)

print(action_0)
print(action_1)

actions =np.reshape(actions, (1,4))
print(actions)
print(actions[:,:2])
print(actions[:,2:])

[[ 0.77472453 -0.06324616]]
[[-0.2808704  -0.60826816]]
[[ 0.77472453 -0.06324616 -0.2808704  -0.60826816]]
[[ 0.77472453 -0.06324616]]
[[-0.2808704  -0.60826816]]


In [22]:
n_episodes = 2000

scores_window = deque(maxlen = 100)
scores_all = []
rolling_average = []


beta_schedule = lambda episode: min(1.0, BETA_START + episode * (1.0)/ n_episodes)
noise_schedule = lambda episode: max(NOISE_END, NOISE_START - episode * ((NOISE_START - NOISE_END) / NOISE_DECAY ))


for i_episode in range(1, n_episodes+1):
    
    beta = beta_schedule(i_episode)
    eps = noise_schedule(i_episode)
    env_info = env.reset(train_mode = True)[brain_name]
    states = env_info.vector_observations
    states = np.reshape(states, (1,48))
    agent_0.reset()
    agent_1.reset()
    scores = np.zeros(num_agents)
    
    while True:
        action_0 = agent_0.act(states, ADD_NOISE)
        action_1 = agent_0.act(states, ADD_NOISE)
        
        actions =np.concatenate((action_0, action_1), axis =0)
        actions =np.reshape(actions, (1,4))
        
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        next_states = np.reshape(next_states, (1,48))
        rewards = env_info.rewards
        dones = env_info.local_done
        
        if np.any(dones):
            done=True
        else:
            done = False
        
        agent_0.step(states, actions, rewards[0], next_states, done, 0, beta)
        agent_1.step(states, actions, rewards[1], next_states, done, 0, beta)
        
        scores += rewards
        states = next_states
        
        if np.any(done):
            break
        
    scores_window.append(np.max(scores))
    scores_all.append(np.max(scores))
    rolling_average.append(np.mean(scores_window))
        
        
    if i_episode % 10 ==0:
        print('Episode {}\tMax Reward: {:.3f}\tAverage Reward: {:.3f}'. format(
            i_episode, np.max(scores), np.mean(scores_window)))
    if np.mean(scores_window)>=0.5:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.3f}').format(
            i_episode, np.max(scores), np.mean(scores_window))
        torch.save(agent_0.actor_local.state_dict(), 'checkpoint_actor_0.pth')
        torch.save(agent_0.actor_local.state_dict(), 'checkpoint_actor_0.pth')
        torch.save(agent_1.actor_local.state_dict(), 'checkpoint__actor_1.pth')
        torch.save(agent_1.critic_local.state_dict(), 'checkpoint_critic_1.pth')
            




Episode 10	Max Reward: 0.000	Average Reward: 0.009
Episode 20	Max Reward: 0.000	Average Reward: 0.010
Episode 30	Max Reward: 0.000	Average Reward: 0.010
Episode 40	Max Reward: 0.090	Average Reward: 0.010
Episode 50	Max Reward: 0.000	Average Reward: 0.008
Episode 60	Max Reward: 0.000	Average Reward: 0.006
Episode 70	Max Reward: 0.000	Average Reward: 0.005
Episode 80	Max Reward: 0.000	Average Reward: 0.005
Episode 90	Max Reward: 0.000	Average Reward: 0.004
Episode 100	Max Reward: 0.000	Average Reward: 0.004
Episode 110	Max Reward: 0.000	Average Reward: 0.005
Episode 120	Max Reward: 0.000	Average Reward: 0.007
Episode 130	Max Reward: 0.000	Average Reward: 0.007
Episode 140	Max Reward: 0.000	Average Reward: 0.008
Episode 150	Max Reward: 0.000	Average Reward: 0.010
Episode 160	Max Reward: 0.000	Average Reward: 0.010
Episode 170	Max Reward: 0.000	Average Reward: 0.011
Episode 180	Max Reward: 0.100	Average Reward: 0.015
Episode 190	Max Reward: 0.100	Average Reward: 0.020
Episode 200	Max Rewar

Episode 1580	Max Reward: 0.100	Average Reward: 0.130
Episode 1590	Max Reward: 0.190	Average Reward: 0.128
Episode 1600	Max Reward: 0.100	Average Reward: 0.120
Episode 1610	Max Reward: 0.100	Average Reward: 0.119
Episode 1620	Max Reward: 0.000	Average Reward: 0.113
Episode 1630	Max Reward: 0.200	Average Reward: 0.115
Episode 1640	Max Reward: 0.000	Average Reward: 0.104
Episode 1650	Max Reward: 0.000	Average Reward: 0.105
Episode 1660	Max Reward: 0.200	Average Reward: 0.103
Episode 1670	Max Reward: 0.200	Average Reward: 0.090
Episode 1680	Max Reward: 0.090	Average Reward: 0.086
Episode 1690	Max Reward: 0.000	Average Reward: 0.087
Episode 1700	Max Reward: 0.100	Average Reward: 0.093
Episode 1710	Max Reward: 0.100	Average Reward: 0.093
Episode 1720	Max Reward: 0.100	Average Reward: 0.093
Episode 1730	Max Reward: 0.000	Average Reward: 0.092
Episode 1740	Max Reward: 0.090	Average Reward: 0.095
Episode 1750	Max Reward: 0.000	Average Reward: 0.095
Episode 1760	Max Reward: 0.100	Average Reward:

In [15]:
torch.from_numpy(np.ones(5))

tensor([ 1.,  1.,  1.,  1.,  1.], dtype=torch.float64)

In [19]:
torch.from_numpy(next_states)

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,  -6.6949,  -1.5000,  -0.0000,   0.0000,
           6.8317,   5.8901,  -0.0000,   0.0000,  -4.1958,  -0.9832,
          24.9906,   6.2152,   6.8317,   5.6939,  24.9906,   6.2152,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,  -6.9963,  -1.5000,   0.0000,   0.0000,
          -6.8317,   5.8901,   0.0000,   0.0000,  -6.5146,  -0.9832,
           4.8171,   6.2152,  -6.8317,   5.6939,   4.8171,   6.2152]], dtype=torch.float64)